# Training Model with V1 Datas

### Initializing GPU

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


2025-07-22 16:16:36.666006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753193796.895092    1005 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753193796.952191    1005 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753193797.387442    1005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753193797.387476    1005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753193797.387478    1005 computation_placer.cc:177] computation placer alr

### Imports

In [2]:

from tensorflow import keras
import numpy as np
import json
from sklearn.model_selection import train_test_split
import os
import  random
import numpy as np
from tensorflow.keras import layers
import keras as keras
from keras.applications import  Xception
from PIL import Image
import random
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)
np.set_printoptions(precision=3, suppress=True)



2.19.0
Num GPUs Available:  1


### Global variables

In [17]:

IS_WSL = True
WINDOWS_DATA_FOLDER="/mnt/e/code/ai/data/tcg/"
GENERATED_DATAS_V1_FOLDER_PATH = "data/generated_datas_v1/"
if IS_WSL :
    GENERATED_DATAS_V1_FOLDER_PATH = WINDOWS_DATA_FOLDER + GENERATED_DATAS_V1_FOLDER_PATH

JSONS_FOLDER=GENERATED_DATAS_V1_FOLDER_PATH+"json/"
IMAGES_FOLDER=GENERATED_DATAS_V1_FOLDER_PATH+"images/"

RANDOM_STATE=42
RESIZE_SIZE=224
RGB = True
EPOCHS = 10
BATCH_SIZE = 28
TEST_SIZE = 0.25
MIN_ASSETS=0
MAX_ASSETS=35000
if RGB:
    RGB_VALUE = 3
else:
    RGB_VALUE = 1



In [13]:

def load_files_list():

    json_files_list = []
    for json_file in os.listdir(JSONS_FOLDER):
        if json_file.endswith(".json"):
            json_files_list.append(json_file)
           

    random.Random(RANDOM_STATE).shuffle(json_files_list)
    json_files_list = json_files_list[MIN_ASSETS:MAX_ASSETS]
    print("Total files : ",json_files_list.__len__())
    print("First file : ",json_files_list[0])
    return json_files_list



### Loading files in generator

In [14]:
def read_json_from_s3(json_key):
    with open(JSONS_FOLDER + json_key) as json_data:
        raw_json = json.load(json_data)
        image_key = IMAGES_FOLDER + raw_json["asset"]["name"]
        points = raw_json["regions"][0]["points"]
        x0 = points[0]["x"]
        y0 = points[0]["y"] 
        x1 = points[1]["x"] 
        y1 = points[1]["y"] 
        x2 = points[2]["x"]
        y2 = points[2]["y"]
        x3 = points[3]["x"]
        y3 = points[3]["y"]
        new_json = {
            "image_key": image_key,
            "bounding_boxes": [x0, y0, x1, y1, x2, y2, x3, y3]
        }
        return new_json
    return None
# Fonction pour lire les images depuis S3
def read_image_from_s3(image_key):
    try:
        img = Image.open(image_key)
        img = img.resize((RESIZE_SIZE, RESIZE_SIZE))
        img = np.asarray( img, dtype="int32" )
        np.array(img)
        img = img/255
        return img
    except Exception as e:
        print(e)
        return None
# Fonction pour créer un dataset TensorFlow
def create_dataset(json_keys, batch_size=32):
    def generator():
        for json_key in json_keys:
            annotations = read_json_from_s3(json_key)
            if annotations == None or annotations["image_key"] == None :
                print("File : " + json_key + " is Malformated, delete it")
                continue
            img = read_image_from_s3(annotations["image_key"])
            if img is None:
                print("Image : " + annotations["image_key"] + " is Malformated, delete it")
                continue
            yield img, annotations['bounding_boxes']
    dataset = tf.data.Dataset.from_generator(generator,
                                             output_signature=(
                                                 tf.TensorSpec(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE), dtype=tf.float32),
                                                 tf.TensorSpec(shape=(8,), dtype=tf.float32)
                                             ))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

### Load datasets

In [18]:
json_files_list = load_files_list()

train_files, val_files = train_test_split(json_files_list, shuffle=True, random_state=42, test_size=TEST_SIZE)
print("Train files : ", len(train_files))
print("Val files : ", len(val_files))
train_dataset = create_dataset(train_files, BATCH_SIZE)
val_dataset = create_dataset(val_files, BATCH_SIZE)


Total files :  35000
First file :  gym1-108_-3_1_-6_584_39.json
Train files :  26250
Val files :  8750


In [16]:

# Définir explicitement la couche d'entrée
input_layer = layers.Input(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE))
xception = Xception(weights="imagenet", include_top=False)(input_layer)

# Construire le reste du modèle
flatten = layers.Flatten(name='flatten')(xception)
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(flatten)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(8, activation='sigmoid', name='bb_head')(locator_branch)

# Créer le modèle
model = tf.keras.Model(inputs=input_layer, outputs=[locator_branch])

# Résumé du modèle
model.summary()

# Définir les pertes et compiler le modèle
losses = {"bb_head": tf.keras.losses.MeanSquaredError()}


model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])




Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_1 (Dense)                    │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_2 (Dense)                    │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_3 (Dense)                    │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_head (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,717,264 (128.62 MB)

 Trainable params: 33,662,736 (128.41 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [21]:
# Callbacks
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/model_checkpoint_{epoch:02d}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=False,
    verbose=1
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='min',
    verbose=1
)

In [24]:

# Entraînement du modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    callbacks=[checkpoint,early_stopping]
)


Epoch 1/3
    938/Unknown 4607s 5s/step - accuracy: 0.9204 - loss: 5.1569e-05

2025-07-23 15:05:28.934577: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16782528912715832793
2025-07-23 15:05:28.936942: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 11095919447368855109



Epoch 1: val_accuracy did not improve from 0.93726
938/938 ━━━━━━━━━━━━━━━━━━━━ 6067s 6s/step - accuracy: 0.9204 - loss: 5.1562e-05 - val_accuracy: 0.9331 - val_loss: 6.5392e-05
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9246 - loss: 3.7794e-05
Epoch 2: val_accuracy improved from 0.93726 to 0.95006, saving model to checkpoints/model_checkpoint_02.keras
938/938 ━━━━━━━━━━━━━━━━━━━━ 5795s 6s/step - accuracy: 0.9246 - loss: 3.7791e-05 - val_accuracy: 0.9501 - val_loss: 5.8078e-05
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9254 - loss: 3.3705e-05
Epoch 3: val_accuracy did not improve from 0.95006
938/938 ━━━━━━━━━━━━━━━━━━━━ 5759s 6s/step - accuracy: 0.9254 - loss: 3.3705e-05 - val_accuracy: 0.9242 - val_loss: 7.4555e-05


2025-07-23 18:42:23.310620: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16782528912715832793


In [25]:
model.save("model-35k-13.keras")